In [1]:
from pyspark.mllib.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import StringIndexer
from pyspark.ml.feature import IndexToString
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml import Pipeline
from pyspark.ml import PipelineModel

In [2]:
# input and output folders
trainingData = "ex_dataCategorical/trainingData.csv"
unlabeledData = "ex_dataCategorical/unlabeledData.csv"
outputPath = "predictionsDTCategoricalPipeline/"

In [3]:
# *************************
# Training step
# *************************

# Create a DataFrame from trainingData.csv
# Training data in raw format
trainingData = spark.read.load(trainingData,\
                     format="csv",\
                     header=True,\
                     inferSchema=True)

In [4]:
#trainingData.show()

In [5]:
# Define an assembler to create a column (features) of type Vector 
# containing the double values associated with columns attr1, attr2, attr3
assembler = VectorAssembler(inputCols=["attr1", "attr2", "attr3"],\
                            outputCol="features")

In [6]:
#assembler.transform(trainingData).show()

In [7]:
# The StringIndexer Estimator is used to map each class label 
# value to an integer value (casted to a double).
# A new attribute called label is generated by applying 
# transforming the content of the categoricalLabel attribute.
labelIndexer = StringIndexer(inputCol="categoricalLabel", outputCol="label", handleInvalid="keep")\
.fit(trainingData)

In [8]:
# Create a DecisionTreeClassifier object.  
# DecisionTreeClassifier is an Estimator that is used to 
# create a classification model based on decision trees.
dt = DecisionTreeClassifier()

In [9]:
# We can set the values of the parameters of the Decision Tree
# For example we can set the measure that is used to decide if a 
# node must be split.
# In this case we set gini index
dt.setImpurity("gini")

DecisionTreeClassifier_4897992eafd8

In [10]:
# At the end of the pipeline we must convert indexed labels back 
# to original labels (from numerical to string).
# The content of the prediction attribute is the index of the predicted class
# The original name of the predicted class is stored in  the predictedLabel 
# attribute.
# IndexToString creates a new column (called predictedLabel in 
# this example) that is based on the content of the prediction column. 
# prediction is a double while predictedLabel is a string
labelConverter = IndexToString(inputCol="prediction", outputCol="predictedLabel", labels=labelIndexer.labels)

In [11]:
# Define a pipeline that is used to create the decision tree
# model on the training data. The pipeline includes also 
# the preprocessing and postprocessing steps
pipeline = Pipeline().setStages([assembler, labelIndexer, dt, labelConverter])

In [12]:
# Execute the pipeline on the training data to build the 
# classification model
classificationModel = pipeline.fit(trainingData)

In [13]:
# Now, the classification model can be used to predict the class label
# of new unlabeled data

In [14]:
# Create a DataFrame from unlabeledData.csv
# Unlabeled data in raw format
unlabeledData = spark.read.load(unlabeledData,\
                     format="csv", header=True, inferSchema=True)

In [15]:
unlabeledData.printSchema()
unlabeledData.show()

root
 |-- categoricalLabel: string (nullable = true)
 |-- attr1: double (nullable = true)
 |-- attr2: double (nullable = true)
 |-- attr3: double (nullable = true)

+----------------+-----+-----+-----+
|categoricalLabel|attr1|attr2|attr3|
+----------------+-----+-----+-----+
|            null| -1.0|  1.5|  1.3|
|            null|  3.0|  2.0| -0.1|
|            null|  0.0|  2.2| -1.5|
+----------------+-----+-----+-----+



In [16]:
# Make predictions on the unlabled data using the transform() method of the 
# trained classification model transform uses only the content of 'features' 
# to perform the predictions. The model is associated with the pipeline and hence
# also the assembler is executed
predictionsDF = classificationModel.transform(unlabeledData)

In [17]:
predictionsDF.printSchema()
predictionsDF.show()

root
 |-- categoricalLabel: string (nullable = true)
 |-- attr1: double (nullable = true)
 |-- attr2: double (nullable = true)
 |-- attr3: double (nullable = true)
 |-- features: vector (nullable = true)
 |-- label: double (nullable = false)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = false)
 |-- predictedLabel: string (nullable = true)

+----------------+-----+-----+-----+--------------+-----+-------------+-------------+----------+--------------+
|categoricalLabel|attr1|attr2|attr3|      features|label|rawPrediction|  probability|prediction|predictedLabel|
+----------------+-----+-----+-----+--------------+-----+-------------+-------------+----------+--------------+
|            null| -1.0|  1.5|  1.3|[-1.0,1.5,1.3]|  2.0|[0.0,1.0,0.0]|[0.0,1.0,0.0]|       1.0|      Positive|
|            null|  3.0|  2.0| -0.1|[3.0,2.0,-0.1]|  2.0|[2.0,0.0,0.0]|[1.0,0.0,0.0]|       0.0|      Negative|
|            null|  

In [18]:
# The returned DataFrame has the following schema (attributes)
# - attr1: double (nullable = true)
# - attr2: double (nullable = true)
# - attr3: double (nullable = true)
# - features: vector (values of the attributes)
# - label: double (value of the class label)
# - rawPrediction: vector (nullable = true)
# - probability: vector (The i-th cell contains the probability that the 
#                        current record belongs to the i-th class
# - prediction: double (the predicted class label)
# - predictedLabel: string (nullable = true)

# Select only the original features (i.e., the value of the original attributes 
# attr1, attr2, attr3) and the predicted class for each record
predictions = predictionsDF.select("attr1", "attr2", "attr3", "predictedLabel")

In [19]:
#predictions.show()

In [20]:
# Save the result in an HDFS output folder
predictions.write.csv(outputPath, header="true")